In [11]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import random

In [20]:
def generate_label(row):
    for name, value in row.items():
        label = random.random()*value
    return label + random.random()
def generate_data(num_features, num_size):
    np.random.seed(42)
    cols = []
    for i in range(1,num_features+1):
        cols.append('X_' + str(i))
    df = pd.DataFrame(np.random.rand(num_size, num_features), columns=cols)
    df['label'] = df.apply(generate_label,axis=1)
    return df
all_df = generate_data(10,1000)

In [27]:
from sklearn.model_selection import train_test_split
def split_data(df,label_column):
    X = df.copy()
    y = df[label_column]
    X.drop(columns=label_column,inplace=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = split_data(all_df,'label')

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
def linear_regression(X_train, X_test, y_train, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    print("Coefficients:", model.coef_)
    print("Intercept:", model.intercept_)

    
    df_feature_imp = pd.DataFrame({'variable':X_train.columns, 'imp':model.coef_})
    print('Feature Importance:\n', df_feature_imp)

    print('R-squared:', model.score(X_test, y_test))
linear_regression(X_train, X_test, y_train, y_test)

Coefficients: [-0.0038409  -0.00628674  0.01026724 -0.07021051 -0.01741712 -0.02010502
 -0.07998465 -0.06486521 -0.04490953  0.43800207]
Intercept: 0.662359201502498
Feature Importance:
   variable       imp
0      X_1 -0.003841
1      X_2 -0.006287
2      X_3  0.010267
3      X_4 -0.070211
4      X_5 -0.017417
5      X_6 -0.020105
6      X_7 -0.079985
7      X_8 -0.064865
8      X_9 -0.044910
9     X_10  0.438002
R-squared: 0.13398565494416736
